In [6]:
import requests

url = "http://localhost:11434/api/chat"

data = {
    "model": "llama3",
    "messages": [
        {"role": "user", "content": "Hello, kitty"}
    ],
    "stream": False,
    "options": {
        "temperature": 0.7,
        "num_predict": -1,
    }
}

response = requests.post(url, json=data)
response.json()

{'model': 'llama3',
 'created_at': '2024-08-09T15:54:11.3689119Z',
 'message': {'role': 'assistant',
  'content': 'Meowwwww! *rubs against leg* Hi there, friend! How can I help or entertain you today?'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 2826758300,
 'load_duration': 2445932900,
 'prompt_eval_count': 13,
 'prompt_eval_duration': 19859000,
 'eval_count': 26,
 'eval_duration': 359662000}

In [5]:
response.json()['message']['content']

"Meow! *rubs against leg* Ah, nice to meet you human. I'm feeling quite purr-fect today. The sunbeams in this room are simply divine. *stretches and arches back* Do you have some treats for me?"

In [1]:
import ast
import re
import json
import time
import requests

url = "http://localhost:11434/api/chat"

path = "training_data/pro"
# suffix_list = ['', '_CoT', '_ToT']
suffix_list = ['_IRSfix', '_IRSori', '_POP']
# _suf = suffix_list[0]
_model_suf = '_llama3'
model_name = 'llama3'

prompt_system_accept = "You are a recommender system. Given the user profile and historical data, analyze the user's interests. Based on this information, would the user be interested in watching the movies in the Influence path step by step? Answer with a probability for a movie between 0 and 1, where 0 means 'definitely not interested' and 1 means 'definitely interested'. If uncertain, make your best guess, then return the mean probability."

prompt_system_relevance = "You are a professional movie critic. Given the Influence path, based on your understanding of the movies, what's the relatedness of each 2 adjacent movies in the influence path? Answer with a probability between 0 and 1, where 1 means 'definitely related' and 0 means 'definitely not related'. If uncertain, make your best guess, then return the mean probability."

system_prompt_list = [[prompt_system_accept, '_accept'], [prompt_system_relevance, '_rel']]

def get_score(_question, _prompt, _system_prompt):
    if _prompt == []:
        return [0, 0]
    else:
        influence_path = str.join("\n", _prompt)

    _question = _question[:_question.index("Target movie:")]
    user_prompt = f"{_question}Influence path:\n{influence_path}"

    # print(user_prompt)

    loop_flag = 0
    while True:
        message = [
            {"role": "system", "content": _system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        data = {
            "model": model_name,
            "messages": message,
            "stream": False
        }
        response = requests.post(url, json=data)
        # print(prompt_system_accept)
        # print(user_prompt)
        # print(response.text)
        
        # gpt_reply = response.json()['message']['content']
        # print(gpt_reply)
        try:
            gpt_reply = response.json()['message']['content']
        except:
            print(f"message: {user_prompt}")
            print(f"Error: {response}")
            continue

        message.append({"role": "assistant", "content": gpt_reply})
        message.append({"role": "user", "content": "Only numerical response required, without any other word. Output the mean probability."})
        
        data = {
            "model": model_name,
            "messages": message,
            "stream": False
        }
        response = requests.post(url, json=data)
        
        # gpt_reply = response.json()['message']['content']
        # print(gpt_reply)
        try:
            gpt_reply = response.json()['message']['content']
        except:
            print(f"message: {user_prompt}")
            print(f"Error: {response}")
            continue
        # print(gpt_reply)

        if re.search('0\.\d+', gpt_reply):
            return [float(re.search('0\.\d+', gpt_reply).group()), gpt_reply]

        loop_flag += 1
        if loop_flag >= 2:
            print(gpt_reply)

for system_prompt, system_prompt_suffix in system_prompt_list:
    print(system_prompt_suffix)
    for _suf in suffix_list:
        print(_suf)
        with open(f"{path}/output_gpt_1m{_suf}_pro.json", "r", encoding='utf-8') as f:
            path_result = json.load(f)
        with open("training_data/train_1m.json", "r", encoding='utf-8') as f:
            training_data_prompt = json.load(f)
        
        k = 1
        print(k)
        all_result = []
        for prefix, influ_path in zip(training_data_prompt['llm'], path_result):
            if k % 20 == 0:
                print(k)
            k += 1
            all_result.append([get_score(prefix[1], influ_path[0], system_prompt), get_score(prefix[1], influ_path[1], system_prompt)])
            # print(all_result)
            # if k == 20:
            #     break
            # break
        print(k)
        
        with open(f"{path}/llm_result_1m{_suf}{system_prompt_suffix}{_model_suf}.json", "w") as outfile:
            json.dump(all_result, outfile)
        
        sum = 0
        count = 0
        for _ in all_result:
        
            a = _[0][0]
            b = _[1][0]
            max_ = max(a, b)
            if max_ == 0:
                continue
            sum += max_
            count += 1
        
        print(sum / count)


_accept
_IRSfix
1
20
40
60
80
100
120
124
0.5804182044715392
_IRSori
1
20
40
60
80
100
120
124
0.5974048779945528
_POP
1
20
40
60
80
100
120
124
0.591758672059621
_rel
_IRSfix
1
20
40
60
80
100
120
124
0.47546549167669355
_IRSori
1
20
40
60
80
100
120
124
0.45836712708601396
_POP
1
20
40
60
80
100
120
124
0.45371633054843014


In [3]:
# Lastfm

import ast
import re
import json
import time
import requests

url = "http://localhost:11434/api/chat"

path = "training_data/lastfm"
# suffix_list = ['', '_CoT', '_ToT']
suffix_list = ['', '_CoT', '_ToT', '_IRSfix', '_IRSori', '_POP']
# _suf = suffix_list[0]
_model_suf = '_llama3'
model_name = 'llama3'

prompt_system_accept = "You are a recommender system. Given the user profile and historical data, analyze the user's interests. Based on this information, would the user be interested in the music artists in the Influence path step by step? Answer with a probability for an artist between 0 and 1, where 0 means 'definitely not interested' and 1 means 'definitely interested'. If uncertain, make your best guess, then return the mean probability."

prompt_system_relevance = "You are a professional music critic. Given the Influence path, based on your understanding of the music artists, what's the relatedness of each 2 adjacent artists in the influence path? Answer with a probability between 0 and 1, where 1 means 'definitely related' and 0 means 'definitely not related'. If uncertain, make your best guess, then return the mean probability."

system_prompt_list = [[prompt_system_accept, '_accept'], [prompt_system_relevance, '_rel']]

def get_score(_question, _prompt, _system_prompt):
    if _prompt == []:
        return [0, 0]
    else:
        influence_path = str.join("\n", _prompt)

    # _question = _question[:_question.index("Target artist:")]
    # user_prompt = f"{_question}Influence path:\n{influence_path}"
    
    user_prompt = f"Historical data:\n{_question}\nInfluence path:\n{influence_path}"
    
    # print(user_prompt)

    loop_flag = 0
    while True:
        message = [
            {"role": "system", "content": _system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        data = {
            "model": model_name,
            "messages": message,
            "stream": False
        }
        response = requests.post(url, json=data)
        # print(prompt_system_accept)
        # print(user_prompt)
        # print(response.text)
        
        # gpt_reply = response.json()['message']['content']
        # print(gpt_reply)
        try:
            gpt_reply = response.json()['message']['content']
        except:
            print(f"message: {user_prompt}")
            print(f"Error: {response}")
            continue

        message.append({"role": "assistant", "content": gpt_reply})
        message.append({"role": "user", "content": "Only numerical response required, without any other word. Output the mean probability."})
        
        data = {
            "model": model_name,
            "messages": message,
            "stream": False
        }
        response = requests.post(url, json=data)
        
        # gpt_reply = response.json()['message']['content']
        # print(gpt_reply)
        try:
            gpt_reply = response.json()['message']['content']
        except:
            print(f"message: {user_prompt}")
            print(f"Error: {response}")
            continue
        # print(gpt_reply)

        if re.search('0\.\d+', gpt_reply):
            return [float(re.search('0\.\d+', gpt_reply).group()), gpt_reply]

        loop_flag += 1
        if loop_flag >= 2:
            print(gpt_reply)

for system_prompt, system_prompt_suffix in system_prompt_list:
    print(system_prompt_suffix)
    for _suf in suffix_list:
        print(_suf)
        with open(f"{path}/output_gpt_lf{_suf}_pro.json", "r", encoding='utf-8') as f:
            path_result = json.load(f)
        with open("training_data/train_lf.json", "r", encoding='utf-8') as f:
            training_data_prompt = json.load(f)
        
        k = 1
        print(k)
        all_result = []
        for prefix, influ_path in zip(training_data_prompt['llm'], path_result):
            if k % 20 == 0:
                print(k)
            k += 1
            all_result.append([get_score(prefix[1], influ_path[0], system_prompt), get_score(prefix[1], influ_path[1], system_prompt)])
            # print(all_result)
            # if k == 20:
            #     break
            # break
        print(k)
        
        with open(f"{path}/llm_result_lf{_suf}{system_prompt_suffix}{_model_suf}.json", "w") as outfile:
            json.dump(all_result, outfile)
        
        sum = 0
        count = 0
        for _ in all_result:
        
            a = _[0][0]
            b = _[1][0]
            max_ = max(a, b)
            if max_ == 0:
                continue
            sum += max_
            count += 1
        
        print(sum / count)


_accept

1
20
40
60
80
100
101
0.660361111111111
_CoT
1
20
40
60
80
100
101
0.6723650332999996
_ToT
1
20
40
60
80
100
101
0.6521161616161615
_IRSfix
1
20
40
60
80
100
101
0.5245683333333333
_IRSori
1
20
40
60
80
100
101
0.5250613333000002
_POP
1
20
40
60
80
100
101
0.5213343333000002
_rel

1
20
40
60
80
100
101
0.6679797979797979
_CoT
1
20
40
60
80
100
101
0.6705203333333334
_ToT
1
20
40
60
80
100
101
0.6574242424242424
_IRSfix
1
20
40
60
80
100
101
0.42959458333333345
_IRSori
1
20
40
60
80
100
101
0.4370898817285985
_POP
1
20
40
60
80
100
101
0.4564230080608774
